# Manipulating meteorological data
### Python workshop, EMS2019

This is **not** xarray tutorial, nor jupyter tutorial, nor visualisation tutorial.  

This part of workshop is about exploring meteorological data from the CDS.

All tools are just that - tools.

Use this notebook as a starting point, and the rest of internet to learn further.

# Importing libraries
First, we're going to import libraries used by this notebook.  
[cdsapi](https://cds.climate.copernicus.eu/api-how-to) - to access the data  
[pandas](https://pandas.pydata.org/pandas-docs/stable/) and [xarray](http://xarray.pydata.org/en/stable/') - to manipulate the data  
[matplotlib]('https://matplotlib.org/') and [seaborn]('https://seaborn.pydata.org/') - for plotting



In [ ]:
import cdsapi
import pandas as pd
import seaborn as sns
import xarray as xr
import xarray.ufuncs as xu
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import date, timedelta

# The data
We will be using already downloaded data. Let's see what is in our directory.  
All weather data is in **data/weather** directory.

Using Jupyter's Built-in magic commands we can access shell commands. They all start with %. More about magic comands [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html).  

In [ ]:
%ls -rtl data/weather/*.nc data/weather/*.grib

## GRIB & netCDF
First we inspect what is in **min_max_t_and_wind_gust.nc** and **min_max_t_and_wind_gust.grib** files.  
We got the netCDF file from the *Example 4 - Hourly ERA5 data in netcdf format* in CDSAPI_examples notebook, and the grib with a modified version of that request.  

They contain the same hourly reanalysis data for 5 days in 2018 for three variables: Wind gusts at 10 m, Minimum and Maximum 2 m temperature.  

Lets open both and see how they are both structured.

NetCDF is pretty straigtforward.

In [ ]:
ds_nc = xr.open_dataset('data/weather/min_max_t_and_wind_gust.nc')
ds_nc

**xarray** can't handle grib files out of the box. For this we need to use [**cfgrib**](https://github.com/ecmwf/cfgrib) as an engine.

In [ ]:
ds_grib = xr.open_dataset('data/weather/min_max_t_and_wind_gust.grib',engine='cfgrib')
ds_grib

We can now see the diferences in the NetCDF and GRIB metadata loaded into xarray dataset.  

There are also diferences in the structure of the dataset.  

- Dataset loaded from netCDF has only one time coordinate and dimension  
- Dataset loaded from GRIB file has time and step.  

This is because these 3 fields don't have instatnatious values, but minimum or maximum over some period. This kind of fields have the type as forecast, and in grib they contain start of the forecast and step as dimensions but also a **valid_time** as coordinate.  
More about this in the [ERA5 documentation](https://confluence.ecmwf.int/pages/viewpage.action?pageId=85402030#ERA5terminology:analysisandforecast;timeandsteps;instantaneousandaccumulatedandmeanratesandmin/maxparameters-'step'andinstantaneous,accumulatedandmin/maxparameters)

We can get more metadata for each variable by inspecting them separately.

In [ ]:
ds_nc.fg10

In [ ]:
ds_grib.fg10

Let's convert to pandas dataframe just to see what our data actually looks like. We will not be doing any calculations using pandas.

In [ ]:
ds_nc.to_dataframe().head(15)

In [ ]:
ds_grib.to_dataframe().head(15)

## Selecting a subarea
Data downloaded from Web download service is always global. Using the CDSAPI we can retrieve smaller area, but if we already have a global field we can select the subarea using the **sel()** method.  
Using the same method we can select any dimension.  We can select only one point or all the points between the interval.

In this example we will select the subarea and one time step so we can plot it to check the area.

In [ ]:
europe = ds_nc.sel(latitude=slice(50,30), longitude=slice(180,240))
europe_one_day = europe.sel(time='2018-05-01T23:00:00')
europe_one_day.fg10.plot()

## Calculating daily data
ERA5 data has hourly data, and we might need to calculate daily mean, minimum and maximum temperatures, or daily sum of precipitation. Xarray has handy method **resample** which does this for us with one line of code.  
However it only works on time dimension, and in the GRIB files from previous example time dimension is actually start of the forecast, which is every day at 6 and 18 UTC. This is why it would not be trivial to resample data from hourly to daily.  
We will start with some simpler dataset.

We will first calculate daily mean of all four parameters in our dataset.

In [ ]:
ds_simple = xr.open_dataset('data/weather/simple_data.nc')
ds_simple

In [ ]:
mean_ds = ds_simple.resample(time='D').mean()
mean_ds

### Resampling individual variables
In reality we might not want to calculate same things for all variables. Let's calculate minimum of T2m.  
And to convert to Celsius on the fly.  

When using resample on individual variables, we need to tell xarray on which dimension to calculate minimum. Otherwise it will calculate it for all dimensions. And we only want it on time, not latitudes or longitudes.

In [ ]:
min_t = ds_simple.t2m.resample(time='D').min('time')-273.15
min_t

It seems like this converted our data to DataArray. Just to to_dataset() to get the data back as Xarray Dataset

In [ ]:
min_t_ds = min_t.to_dataset()
min_t_ds

We should check if this did what we wanted. We should get the same thing if we now select one point and check the data as if we first select the point and then do resampling.

In [ ]:
A = min_t_ds.sel(latitude=51.5, longitude = 0.12, method='nearest')
A.to_dataframe()

In [ ]:
B = ds_simple.t2m.sel(latitude=51.5, longitude = 0.12, method='nearest').resample(time='D').min()-273.15
B.to_dataframe()

Now that we know what we're doing, we can calculate the maximum of wind speed.

But we need to calculate wind speed first.

In [ ]:
u = ds_simple.u10
v = ds_simple.v10
speed = xu.sqrt(u**2+v**2)

And now we can calculate daily maximum speed and convert it back to dataset. 
We are renaming the resulting DataArray in order to convert it to dataset.

In [ ]:
daily_max_speed = speed.resample(time='D').max('time')
daily_max_speed = daily_max_speed.rename('speed')
daily_max_speed_ds = daily_max_speed.to_dataset()
daily_max_speed_ds

## But... accumulated, min/max and mean parameters are different

Remember, values for these parameters are not at the specific time, but acumulated between two steps. Midnight is the first time in the day, and it contains the data from the last hour in the previous day.  
This means, when calculating daily statistics using **'D'** it will belong to the wrong day.

We can correct this by removing the data for the first time step, at midnight on the first day. It belongs to previous day and we don't need it. From then, instead of using 'D' as frequency, we can use '24H'.  

Using **where()** method we can filter out the first 00 step. Where() is very usefuly for filtering data. You can find more info in the [xarray documentation](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.where.html).

In [ ]:
condition = ds_nc.time > ds_nc.time[0]
ds_nc_modified = ds_nc.where(condition, drop=True)

In [ ]:
B = ds_nc_modified.mn2t.sel(latitude=51.5, longitude = 0.12, method='nearest').resample(time='24H').min()-273.15
B.to_dataframe()

## More resampling examples
Some of the frequencies we can use with resample() method are:  

   * ‘AS’: year start
   * ‘QS-DEC’: quarterly, starting on December 1
   * ‘MS’: month start
   * ‘D’: day
   * ‘H’: hour
   * ‘Min’: minute


Try out different time frequencies. You can use 'D' for daily, 'M' for monthly, 'Q' for quarterly, but also '6H' to compute mean over 6 hour periods.

In [ ]:
weekly_t2m = t2m.resample(time='W').mean()-273.15
weekly_t2m.plot()

### Monthly to  quarterly resampling
CDS offers already calculated monthly statistics from ERA5 Dataset, in the **ERA5 monthly averaged data on single levels from 1979 to present** and **ERA5 monthly averaged data on pressure levels from 1979 to present** datasets. That is where our 'monthly_data.grib' file is coming from.   

In [ ]:
monthly_ds = xr.open_dataset('data/weather/monthly_data.grib',engine='cfgrib')
monthly_ds

Our data is monthly and we might want to match it to non-meteorological data that has quarterly interval.  

For example if we want seasonal means (DJF, MAM, JJA, and SON) we can use 'QS-DEC', meaning **quarterly starting with december**.  
Note: This method doesn't seem to take into account that months have different number of days. To find how to calculate proper weighted means, have a look at this example [here](https://github.com/pydata/xarray/blob/master/examples/xarray_seasonal_means.ipynb).

In [ ]:
monthly_t2m = monthly_ds.t2m

In [ ]:
quarterly_t2m = monthly_t2m.sel(latitude=51.5, longitude = 0.12, method='nearest').resample(time='QS-DEC').mean()-273.15
quarterly_t2m.to_dataframe()

We can use resample method to interpolate data too. We need to specify which method to use for the interpolation. Lets interpolate our London monthly data to daily intervals.  
We convert to Celsius on the fly too.

In [ ]:
daily_t2m = monthly_t2m.sel(latitude=51.5, longitude = 0.12, method='nearest').resample(time='D').interpolate('linear')-273.15
daily_t2m.plot()

In [ ]:
daily_t2m.to_dataframe().head(10)

### Some datasets are different....

Let's see what the data from new **ERA5 Land** dataset looks like. In GRIB format we can notice big difference compared to ERA5 data. All the fields have time and step dimensions. In ERA5 this was the case only with accumulated and min/max parameters.   
ERA5 Land is made to be similar to ERA Interim. So here we have start every day at 00 and then 24 'forecast hours'.  

It makes life much easier when resampling to daily data.  

In [ ]:
ds2 = xr.open_dataset('data/weather/bicycle_london_data_2012.grib', engine='cfgrib')
ds2

We will select the point in London to check what the data looks like.

In [ ]:
tp = ds2.tp
tp_london = tp.sel(latitude=51.5, longitude = 0.12, method='nearest')*1000
tp_london.to_dataframe().head(50)

This is actually very convinient if we want daily precipitation, because we don't need to calculate anything. Just select te data for every midnight time step.  
If we have a look at time/step structure, we can see that we need the data with the step 24, or one day. 

In [ ]:
daily_tp_london = tp_london.sel(step=timedelta(1))
daily_tp_london.to_dataframe()

Now we can introduce a new method.

## Rolling

Let's say that every day we need a sum of precipitation for the last N days. Xarray has a method for calculating this: **rolling()**.   

time = 7 means we are doing the sum of previous 7 days.

The data array we got using this method doesn't have a name. It also has a NaN value at the end. This is because we tried to get the 24th step of 31st December which is not there.
To make our data a bit more clean we will drop NaN values and rename our DataArray.

In [ ]:
tp_day_week = daily_tp_london.rolling(time=7).sum()
tpdw = tp_day_week.rename('tp').dropna('time')
tpdw

#### Why did we even care if our DataArray has a name? 

Because unnamed DataArray can not be converted to pandas dataframe.

In [ ]:
tpdw.to_dataframe().head(20)

### Ensemble data

ERA5 dataset offfers 10 members ensemble data on lower spatial resolution (0.5x0.5). 
In our grib file we have this ensemble data: temperature and relative humidity on 2 pressure levels (300 and 500 hPa)

In [ ]:
ds_en = xr.open_dataset('data/weather/ensemble_data_pl.grib',engine = 'cfgrib')
ds_en

Calculate ensemble mean and standard deviation.

In [ ]:
ens_mean = ds_en.mean(dim='number')
ens_std = ds_en.std(dim='number')

Calculate probability that temperature at 500 hPa will be lower than -20C.  

- First we select temperature at 500 hPa and convert to C.  
- Then we make a mask using **where()**. Wherever the values are smaller than -20, the value becomes one, and anything above -20 becomes 0.  
- All we have to do now is sum along all ensemble members, divide by number of members and multiply by 100.

In [ ]:
t_500 = ds_en.t.sel(isobaricInhPa=500)-273.15
mask = t_500.where(t_500<-20, 1).where(t_500>-20,0)
probability = mask.sum(dim='number')*10
probability.sel(time='2018-05-07T21:00:00').plot()

### Further reading

There are many more interesting methods for processing meteorological data.   
A few worth mentioning:
- [diff()](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.diff.html#xarray.Dataset.diff) - creates difference between data over some dimension, useful for calculating pressure tendency
- [groupby()](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.groupby.html) - groups by data on different dimensions. Similar to resample, but works on any dimension
- [quantile()](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.quantile.html) - Compute the qth quantile of the data along the specified dimension.